# 0. Imports

In [105]:
import pandas as pd
import numpy as np
import plotly.express as px


#Suppress scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)


# 1. Extraction

In [106]:
data = pd.read_csv('datasets/kc_house_data.csv')

# 2. Transformation

### Perform data cleansing.

In [107]:
data = data[data['price'] < 500000000]

### Calculate the median price by zip code.

In [108]:
df_zipcode_median = data[['price','zipcode']].groupby('zipcode').median().reset_index()
df_zipcode_median.columns = ['zipcode','price_median']

### Join the column of median prices with the main dataset.

In [109]:
df_recommendations_price = pd.merge(data, df_zipcode_median, on = 'zipcode', how = 'inner')

### Generate the status column with buying recommendation.

In [110]:
df_recommendations_price['status'] = df_recommendations_price.apply( lambda row : 'buy' if (row['condition'] == 5) & 
                                                                                       (row['price'] < row['price_median']) 
                                                                                    else 'not buy', axis=1)


### Generate a "selling_price" with the selling price calculated.

In [111]:
def define_selling_price(table_row):

    current_price = table_row['price']
    median_price = table_row['price_median']

    if (current_price > median_price):
        return current_price*1.1
    else:
        return current_price*1.3

In [112]:
df_recommendations_price['selling_price'] = df_recommendations_price.apply(define_selling_price, axis=1)

# 3. Exploratory Data Analysis

### **Hypothesis 1:** Properties with view to water are 30% more expensive.

### **False**: As observed, properties with water front are more than 30% more expensive.

In [113]:
df_properties_water = df_recommendations_price[['waterfront','price']].groupby('waterfront').mean().reset_index()
df_properties_water.columns = ['waterfront', 'price_mean']
df_properties_water['waterfront_status'] = df_properties_water['waterfront'].apply(lambda wfront : 'yes' if wfront == 1.0 else 'no')


In [114]:
fig = px.bar(df_properties_water, x='waterfront_status', y='price_mean')
fig.show()

### **Hypothesis 2:** Properties with construction date less than 1955 are 50% cheaper on average.

In [115]:
df_properties_1955 = df_recommendations_price[['yr_built','price']].copy()
df_properties_1955['before_1955'] = df_properties_1955['yr_built'].apply(lambda yr_built : 'yes' if(yr_built < 1955) else 'no')
df_properties_1955 = df_properties_1955[['before_1955', 'price']].groupby('before_1955').mean().reset_index()

In [119]:
fig = px.bar(df_properties_1955, x = 'before_1955', y = 'price')
fig.show()

### **Hypothesis 3:** Properties without basement are 50% bigger in lot size than ones without basement.

In [126]:
df_properties_basement = df_recommendations_price[['sqft_lot','sqft_basement']].copy()
df_properties_basement['have_basement'] = df_properties_basement['sqft_basement'].apply(lambda basement : 
                                                                                                'yes' if(basement > 0) 
                                                                                                       else 'no')
df_properties_basement = df_properties_basement[['have_basement', 'sqft_lot']].groupby('have_basement').mean().reset_index()

In [134]:
fig = px.bar(df_properties_basement, x = 'have_basement', y = 'sqft_lot', 
                labels = {'sqft_lot' : 'Mean sqft lot', 'have_basement' : 'Property have basement?'},
                width = 600,
                height = 371)
fig.show()